In [1]:
import pandas as pd 
import numpy as np 

import re
import matplotlib.pyplot as plt 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns 

from tensorflow.keras.layers import Input , Dense , Embedding, LSTM,GlobalMaxPool1D
from tensorflow.keras.models import Model

2025-06-02 14:51:57.301926: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-02 14:51:57.314247: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-02 14:51:57.340910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748856117.382645   13190 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748856117.393625   13190 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748856117.432744   13190 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
df= pd.read_csv('twitter_training.csv')
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [3]:
df.columns = ['id','area','target','text']

In [4]:
df.head()

,id,area,target,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [5]:
df['target'].value_counts()

target
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

In [59]:
df['target'] = df['target'].replace({'Positive':1,'Negative':2,'Neutral':3,'Irrelevant':4})

In [31]:
df.isnull().sum()

id        0
area      0
target    0
text      0
dtype: int64

In [8]:
df.dropna(inplace=True)

In [32]:
def text_cleaning(text):
    text = re.sub(r'@[A-Za-z0-9]+',"",text)
    text = re.sub(r'@[A-Za-zA-Z0-9]+',"",text)
    text = re.sub(r'@[A-Za-z]+',"",text)
    text= re.sub(r'@[-)]+',"",text)
    text = re.sub(r'#',"",text)
    text = re.sub(r'RT[\s]+',"",text)
    text = re.sub(r'https?\/\/\S+',"",text)
    text = re.sub(r'&[a-z;]+','',text)

    return text

In [33]:
df['text'] = df['text'].apply(text_cleaning)
df.head()

,id,area,target,text
0,2401,Borderlands,1,i am coming to the border and i will kill you ...
1,2401,Borderlands,1,im getting on borderland and i will kill you a...
2,2401,Borderlands,1,im coming on borderland and i will murder you ...
3,2401,Borderlands,1,im getting on borderland 2 and i will murder y...
4,2401,Borderlands,1,im getting into borderland and i can murder yo...


In [34]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('all')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

def clean_and_lemmatize(text):
    tokens = word_tokenize(text.lower())
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmatized)

In [35]:
df['text'] = df['text'].apply(clean_and_lemmatize)

In [36]:
X = df['text']
y = df['target']

In [37]:
x_train, x_test, y_train ,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [38]:
tokenizer = Tokenizer(num_words=10000,oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

In [39]:
# Checking the words index and find out the vocab of the data set 
words = tokenizer.word_index
length = len(words)
print("siz of data set vocab is : ", length)

siz of data set vocab is :  28438


In [40]:
train_seq = tokenizer.texts_to_sequences(x_train)
test_seq = tokenizer.texts_to_sequences(x_test)

print("Training Sequence : ", train_seq[0])
print("Testing Sequence : ", test_seq[0])

Training Sequence :  [3, 106, 4, 1019, 15, 1965, 6, 3, 81, 5948, 6, 1, 38, 106, 5, 859, 133, 111, 4, 1187, 1, 2566, 6, 4, 1, 626, 6, 93, 33, 4, 1838, 8, 2566, 3455, 30, 75, 2785, 73, 23, 2834, 60, 2296, 20, 59, 1629, 26, 1187, 1465, 2297, 76, 60, 36, 7, 33, 493, 1045, 895]
Testing Sequence :  [35, 13, 172, 13, 43, 1301, 23]


In [41]:
pad_train = pad_sequences(train_seq)
l = pad_train.shape[1]
print("the length of training sequence is : ",l)

the length of training sequence is :  166


In [42]:
# Padding the test sequence 
pad_test =pad_sequences(test_seq, maxlen=l)
print("The length of test sequence is : ",pad_test.shape[1])

The length of test sequence is :  166


In [ ]:
D = 20 
i = Input(shape = (l ,))
x = Embedding(length+1, D)(i)
x = LSTM(64, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(64,activation='relu')(x)
x = Dense(5,activation='softmax')(x)

model = Model(i,x)

In [49]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [54]:
# Model Training 
model.fit(pad_train,y_train, validation_data=(pad_test, y_test), epochs=3)

Epoch 1/3
1850/1850 ━━━━━━━━━━━━━━━━━━━━ 194s 105ms/step - accuracy: 0.9203 - loss: 0.2178 - val_accuracy: 0.8307 - val_loss: 0.5415
Epoch 2/3
1850/1850 ━━━━━━━━━━━━━━━━━━━━ 206s 107ms/step - accuracy: 0.9296 - loss: 0.1876 - val_accuracy: 0.8392 - val_loss: 0.5544
Epoch 3/3
1850/1850 ━━━━━━━━━━━━━━━━━━━━ 191s 103ms/step - accuracy: 0.9404 - loss: 0.1587 - val_accuracy: 0.8434 - val_loss: 0.5846


In [73]:
text_pred = input("Enter the text : ")
re_text = text_cleaning(text_pred)
cleaned = clean_and_lemmatize(re_text)
seq = tokenizer.texts_to_sequences([cleaned])
padded = pad_sequences(seq,maxlen=l)
pred = model.predict(padded)

predicted_class = pred.argmax(axis=1)[0]  # if using softmax
print(f"Predicted class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Predicted class: 2


In [56]:
# Predict probabilities for test data
y_pred_probs = model.predict(pad_test)

# Convert to class labels
y_pred_classes = y_pred_probs.argmax(axis=1)


463/463 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step


In [57]:
from sklearn.metrics import classification_report, accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Accuracy: 0.843367795121292
              precision    recall  f1-score   support

           1       0.84      0.84      0.84      4123
           2       0.89      0.85      0.87      4463
           3       0.82      0.86      0.84      3589
           4       0.80      0.82      0.81      2624

    accuracy                           0.84     14799
   macro avg       0.84      0.84      0.84     14799
weighted avg       0.84      0.84      0.84     14799



In [58]:
df['target'].value_counts()

target
2    22358
1    20654
3    18108
4    12875
Name: count, dtype: int64

In [72]:
df['text'][25]

'the biggest disappointment of my life came a year ago .'

In [74]:
df.head()

,id,area,target,text
0,2401,Borderlands,1,i am coming to the border and i will kill you ...
1,2401,Borderlands,1,im getting on borderland and i will kill you a...
2,2401,Borderlands,1,im coming on borderland and i will murder you ...
3,2401,Borderlands,1,im getting on borderland 2 and i will murder y...
4,2401,Borderlands,1,im getting into borderland and i can murder yo...
